In [1]:
# Установка официального клиента Elasticsearch
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.6/963.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 1.5 MB/s eta 0:00:00


In [ ]:
from elasticsearch import Elasticsearch
import getpass # Для скрытого ввода ключа (опционально)

# 1. Настройка соединения
# Вставьте ваш Cloud ID (из консоли Elastic Cloud)
CLOUD_ID = "________________________"

# Вставьте ваш API Key
API_KEY = "__________________________"

# 2. Инициализация клиента
es = Elasticsearch(
    cloud_id=CLOUD_ID,
    api_key=API_KEY
)

# 3. Проверка связи
try:
    info = es.info()
    print(f"✅ Успешное подключение к: {info['name']}")
    print(f"📡 Версия Elasticsearch: {info['version']['number']}")
except Exception as e:
    print(f"❌ Ошибка подключения: {e}")

✅ Успешное подключение к: instance-0000000000
📡 Версия Elasticsearch: 9.2.3


In [5]:
index_name = "shop_products"

# Определяем настройки индекса
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},       # Текст: будет анализироваться (разбиваться на слова)
            "description": {"type": "text"}, # Текст: для поиска по описанию
            "price": {"type": "float"},      # Число: для фильтрации по цене
            "category": {"type": "keyword"}  # Ключевое слово: для точных совпадений (категории)
        }
    }
}

# Удаляем индекс, если он уже был (чтобы не было ошибок при перезапуске урока)
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

# Создаем новый индекс
es.indices.create(index=index_name, body=mapping)
print(f"🗂 Индекс '{index_name}' успешно создан.")

🗂 Индекс 'shop_products' успешно создан.


In [6]:
# Подготовим список товаров
products = [
    {
        "title": "Apple MacBook Pro 14",
        "description": "Мощный ноутбук для профессионалов на чипе M3. Идеален для кода.",
        "price": 250000,
        "category": "Laptop"
    },
    {
        "title": "Игровая мышь Logitech",
        "description": "Беспроводная мышь с RGB подсветкой и высокой точностью.",
        "price": 5000,
        "category": "Accessories"
    },
    {
        "title": "Samsung Galaxy S24",
        "description": "Смартфон с отличной камерой и ярким экраном.",
        "price": 120000,
        "category": "Smartphone"
    },
    {
        "title": "Чехол для MacBook",
        "description": "Кожаный чехол для ноутбука Apple, цвет черный.",
        "price": 3000,
        "category": "Accessories"
    }
]

# Загружаем товары в индекс
print("📤 Загрузка данных...")
for product in products:
    resp = es.index(index=index_name, document=product, refresh=True)
    print(f"Добавлен товар: {product['title']} (ID: {resp['_id']})")

📤 Загрузка данных...
Добавлен товар: Apple MacBook Pro 14 (ID: 8zGAXJsBVRLIsG-6Ax47)
Добавлен товар: Игровая мышь Logitech (ID: 9DGAXJsBVRLIsG-6Ax5s)
Добавлен товар: Samsung Galaxy S24 (ID: 9TGAXJsBVRLIsG-6Ax6R)
Добавлен товар: Чехол для MacBook (ID: 9jGAXJsBVRLIsG-6Ax68)


In [7]:
# Поисковый запрос пользователя
user_query = "ноутбук apple"

# Формирование запроса к Elasticsearch
search_body = {
    "query": {
        "multi_match": {
            "query": user_query,
            "fields": ["title^3", "description"], # Символ ^3 повышает важность Заголовка
            "fuzziness": "AUTO" # Включает поиск с учетом опечаток
        }
    }
}

# Выполнение запроса
response = es.search(index=index_name, body=search_body)

# Вывод результатов
print(f"\n🔍 Результаты поиска по запросу: '{user_query}'")
print(f"Найдено документов: {response['hits']['total']['value']}\n")

for hit in response['hits']['hits']:
    score = hit['_score'] # Релевантность (насколько хорошо подходит)
    source = hit['_source'] # Сами данные
    print(f"★ {score:.2f} | {source['title']}")
    print(f"   — {source['description']}")
    print(f"   — Цена: {source['price']} руб.\n")


🔍 Результаты поиска по запросу: 'ноутбук apple'
Найдено документов: 2

★ 3.30 | Apple MacBook Pro 14
   — Мощный ноутбук для профессионалов на чипе M3. Идеален для кода.
   — Цена: 250000 руб.

★ 2.36 | Чехол для MacBook
   — Кожаный чехол для ноутбука Apple, цвет черный.
   — Цена: 3000 руб.



In [8]:
# Задача: Найти товары, в описании которых упоминается "ноутбук"
query = {
    "query": {
        "match": {
            "description": "ноутбук для работы"
            # Elasticsearch сам уберет стоп-слова ("для") и найдет корень слова "ноутбук"
        }
    }
}

response = es.search(index="shop_products", body=query)

print(f"Найдено: {response['hits']['total']['value']}")
for hit in response['hits']['hits']:
    print(f"- {hit['_source']['title']} (Score: {hit['_score']:.2f})")

Найдено: 2
- Apple MacBook Pro 14 (Score: 1.98)
- Чехол для MacBook (Score: 0.73)


In [9]:
# Задача: Найти ноутбуки дороже 50 000 руб.
# При этом товары Apple должны быть выше в списке.
query = {
    "query": {
        "bool": {
            # Обязательное условие (влияет на очки)
            "must": [
                {"match": {"description": "ноутбук"}}
            ],
            # Жесткий фильтр (отсекает лишнее, не влияет на очки)
            "filter": [
                {"range": {"price": {"gte": 50000, "lte": 500000}}}
            ],
            # Пожелание (если есть "Apple" в названии — поднять выше)
            "should": [
                {"match": {"title": "Apple"}}
            ],
            "minimum_should_match": 0 # should не обязателен, если есть must
        }
    },
    "size": 5 # Вернуть только 5 лучших
}

response = es.search(index="shop_products", body=query)

print("--- Результаты сложного поиска ---")
for hit in response['hits']['hits']:
    print(f"[{hit['_score']:.2f}] {hit['_source']['title']} - {hit['_source']['price']} руб.")

--- Результаты сложного поиска ---
[2.19] Apple MacBook Pro 14 - 250000 руб.


In [10]:
query = {
    "query": {
        "multi_match": {
            "query": "macbook pro",
            "fields": [
                "title^3",    # Совпадение в заголовке в 3 раза важнее!
                "description" # Совпадение в описании имеет стандартный вес
            ],
            "type": "best_fields"
        }
    }
}

response = es.search(index="shop_products", body=query)
# Вывод результатов аналогичен предыдущим примерам...

In [11]:
query = {
    "query": {
        "fuzzy": {
            "title": {
                "value": "makbook",  # Намеренная опечатка (k вместо c)
                "fuzziness": "AUTO"  # Автоматическое определение допустимого кол-ва ошибок
            }
        }
    }
}

response = es.search(index="shop_products", body=query)
print(f"Найдено с опечаткой: {response['hits']['total']['value']}")

Найдено с опечаткой: 2


In [12]:
# Задача: Посчитать среднюю цену товаров в каждой категории
query = {
    "size": 0,  # size=0 значит "не возвращай сами товары, только цифры статистики"
    "aggs": {
        "my_categories": { # Произвольное имя агрегации
            "terms": {
                "field": "category", # Группировка по полю category
                "size": 10
            },
            "aggs": { # Вложенная агрегация внутри каждой категории
                "avg_price": {"avg": {"field": "price"}},
                "min_price": {"min": {"field": "price"}},
                "max_price": {"max": {"field": "price"}}
            }
        }
    }
}

response = es.search(index="shop_products", body=query)

print("--- Статистика по категориям ---")
# Разбираем ответ (buckets — это "корзины" или группы)
for bucket in response['aggregations']['my_categories']['buckets']:
    cat_name = bucket['key']
    count = bucket['doc_count']
    avg = bucket['avg_price']['value']
    print(f"Категория: {cat_name:15} | Товаров: {count} | Ср. цена: {avg:.0f} руб.")

--- Статистика по категориям ---
Категория: Accessories     | Товаров: 2 | Ср. цена: 4000 руб.
Категория: Laptop          | Товаров: 1 | Ср. цена: 250000 руб.
Категория: Smartphone      | Товаров: 1 | Ср. цена: 120000 руб.


In [14]:
# Имя индекса
suggest_index = "shop_suggestions"

# 1. Удаляем старый индекс (если есть), чтобы начать с чистого листа
if es.indices.exists(index=suggest_index):
    es.indices.delete(index=suggest_index)

# 2. Создаем индекс с маппингом
# Используем специальный тип поля 'completion' для мгновенных подсказок
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "suggest": {
                "type": "completion",  # Оптимизированная структура для автокомплита
                "analyzer": "simple",
                # Contexts позволяет фильтровать подсказки (например, по категории)
                "contexts": [
                    {"name": "category_ctx", "type": "category"}
                ]
            }
        }
    }
}

es.indices.create(index=suggest_index, body=mapping)
print(f"✅ Индекс '{suggest_index}' создан.")

# 3. Подготовка данных
# Поле 'input' содержит варианты того, как пользователь может искать товар
dataset = [
    {
        "title": "Apple MacBook Pro 14",
        "suggest": {
            "input": ["macbook", "apple mac", "mac pro", "ноутбук apple"],
            "weight": 10, # Вес 10: будет выше в списке
            "contexts": {"category_ctx": "laptops"}
        }
    },
    {
        "title": "Apple Mac Mini",
        "suggest": {
            "input": ["mac mini", "mac station"],
            "weight": 5,
            "contexts": {"category_ctx": "desktops"}
        }
    },
    {
        "title": "Помада MAC Lipstick", # Товар из другой категории (ловушка)
        "suggest": {
            "input": ["mac lipstick", "lipstick"],
            "weight": 5,
            "contexts": {"category_ctx": "beauty"}
        }
    }
]

# Индексируем товары
print("📤 Загрузка данных...")
for doc in dataset:
    es.index(index=suggest_index, document=doc)

# ВАЖНО: Принудительно обновляем индекс
# Без этого поиск может вернуть пустоту, так как данные еще не успели "примениться"
es.indices.refresh(index=suggest_index)

# 4. Выполнение запроса (Эмуляция ввода пользователя)
user_input = "mac"
current_category = "laptops"

print(f"\n⌨️  Пользователь вводит: '{user_input}' (фильтр по категории: '{current_category}')")

suggest_query = {
    "suggest": {
        "my-completion": { # Произвольное имя блока подсказок
            "prefix": user_input, # То, что ввел пользователь
            "completion": {
                "field": "suggest",
                "size": 5, # Максимум 5 подсказок
                "skip_duplicates": True,
                # Фильтруем: искать только среди ноутбуков!
                "contexts": {
                    "category_ctx": current_category
                }
            }
        }
    }
}

response = es.search(index=suggest_index, body=suggest_query)

# 5. Вывод результатов
options = response['suggest']['my-completion'][0]['options']

print("💡 Результат автодополнения:")
if not options:
    print("Ничего не найдено.")
else:
    for opt in options:
        # text — это совпавшая фраза из поля input
        # _source.title — это полное название товара
        print(f" — {opt['text']} (Товар: {opt['_source']['title']})")

✅ Индекс 'shop_suggestions' создан.
📤 Загрузка данных...

⌨️  Пользователь вводит: 'mac' (фильтр по категории: 'laptops')
💡 Результат автодополнения:
 — mac pro (Товар: Apple MacBook Pro 14)


Реализация политики ILM

In [15]:
# Определение политики жизненного цикла
ilm_policy = {
    "phases": {
        # 1. HOT: Активная запись.
        # Здесь данные пишутся и часто ищутся.
        "hot": {
            "actions": {
                "rollover": {
                    # Если индексу исполнился 1 день ИЛИ он весит > 50ГБ,
                    # создаем новый пустой индекс для записи, а этот отправляем в архив.
                    "max_age": "1d",
                    "max_size": "50gb"
                }
            }
        },
        # 2. WARM: Только чтение.
        # Данные перемещаются сюда через 7 дней.
        "warm": {
            "min_age": "7d",
            "actions": {
                # Оптимизация: уменьшаем число шардов и сегментов для экономии места
                "shrink": {"number_of_shards": 1},
                "forcemerge": {"max_num_segments": 1}
            }
        },
        # 3. DELETE: Удаление.
        # Через 30 дней данные удаляются автоматически.
        "delete": {
            "min_age": "30d",
            "actions": {
                "delete": {}
            }
        }
    }
}

# Применение политики к кластеру
# Обратите внимание: в клиенте v8+ используем аргумент 'policy', а не 'body'
response = es.ilm.put_lifecycle(name="logs-policy", policy=ilm_policy)

print(f"✅ Политика ILM создана: {response['acknowledged']}")

✅ Политика ILM создана: True


Приложение А. Решение практической работы (Поисковый сервис)

In [18]:
from elasticsearch import Elasticsearch
from typing import List, Dict, Optional
from dataclasses import dataclass

@dataclass
class SearchResult:
    id: str
    title: str
    score: float
    highlights: Dict[str, List[str]]

class ProductSearchService:
    """
    Поисковый сервис для каталога товаров.
    Демонстрирует best practices работы с Elasticsearch.
    """

    def __init__(self, cloud_id: str, api_key: str, index_name: str = "shop_products"):
        self.es = Elasticsearch(
            cloud_id=cloud_id,
            api_key=api_key
        )
        self.index_name = index_name

        # Проверка соединения
        try:
            info = self.es.info()
            print(f"✅ Сервис подключен к: {info['name']}")
        except Exception as e:
            print(f"❌ Ошибка подключения: {e}")

    def recreate_index(self):
        """
        Создает индекс с правильным маппингом.
        ВАЖНО: Удаляет старый индекс при вызове!
        """
        # Удаляем, если есть (чтобы применить новые настройки)
        self.es.options(ignore_status=[400, 404]).indices.delete(index=self.index_name)

        # Настройки индекса
        mapping = {
            "mappings": {
                "properties": {
                    "title": {"type": "text"},
                    "description": {"type": "text"},
                    "price": {"type": "float"},
                    "category": {"type": "keyword"},
                    # СПЕЦИАЛЬНОЕ ПОЛЕ ДЛЯ АВТОДОПОЛНЕНИЯ
                    "suggest": {
                        "type": "completion",
                        "analyzer": "simple",
                        "preserve_separators": True,
                        "preserve_position_increments": True,
                        "max_input_length": 50
                    }
                }
            }
        }

        self.es.indices.create(index=self.index_name, body=mapping)
        print(f"♻️ Индекс '{self.index_name}' пересоздан с поддержкой автодополнения.")

    def search(self, query: str, category: Optional[str] = None,
               min_price: Optional[float] = None, max_price: Optional[float] = None,
               page: int = 1, size: int = 20) -> Dict:
        """Полнотекстовый поиск."""
        must_clauses = []
        filter_clauses = []

        if query:
            must_clauses.append({
                "multi_match": {
                    "query": query,
                    "fields": ["title^3", "description", "tags^2"],
                    "fuzziness": "AUTO"
                }
            })
        else:
            must_clauses.append({"match_all": {}})

        if category:
            filter_clauses.append({"term": {"category": category}})

        if min_price is not None or max_price is not None:
            price_range = {}
            if min_price is not None: price_range["gte"] = min_price
            if max_price is not None: price_range["lte"] = max_price
            filter_clauses.append({"range": {"price": price_range}})

        body = {
            "query": {
                "bool": {
                    "must": must_clauses,
                    "filter": filter_clauses
                }
            },
            "highlight": {
                "fields": {"title": {}, "description": {}}
            },
            "aggs": {
                "categories": {"terms": {"field": "category", "size": 10}}
            },
            "from": (page - 1) * size,
            "size": size
        }

        response = self.es.search(index=self.index_name, body=body)

        return {
            "total": response['hits']['total']['value'],
            "results": [
                SearchResult(
                    id=hit['_id'],
                    title=hit['_source']['title'],
                    score=hit['_score'],
                    highlights=hit.get('highlight', {})
                ) for hit in response['hits']['hits']
            ],
            "facets": {
                "categories": [
                    {"name": b['key'], "count": b['doc_count']}
                    for b in response['aggregations']['categories']['buckets']
                ]
            }
        }

    def autocomplete(self, prefix: str, limit: int = 5) -> List[str]:
        """Автодополнение."""
        body = {
            "suggest": {
                "product-suggest": {
                    "prefix": prefix,
                    "completion": {
                        "field": "suggest", # Ссылка на поле с типом completion
                        "size": limit,
                        "skip_duplicates": True
                    }
                }
            }
        }

        try:
            response = self.es.search(index=self.index_name, body=body)
            options = response['suggest']['product-suggest'][0]['options']
            return [option['text'] for option in options]
        except Exception as e:
            print(f"Ошибка автодополнения: {e}")
            return []

    def index_product(self, product: Dict):
        """Индексирование товара."""
        # Генерация данных для автодополнения
        # Мы добавляем в подсказки заголовок и отдельные слова из него
        suggest_inputs = [product['title']]
        # Можно добавить теги, если они есть: suggest_inputs.extend(product.get('tags', []))

        product['suggest'] = {
            "input": suggest_inputs
        }

        self.es.index(index=self.index_name, document=product)


# --- БЛОК ТЕСТИРОВАНИЯ ---
if __name__ == '__main__':
    # 1. КОНФИГУРАЦИЯ
    MY_CLOUD_ID = "b9166fa8348b4a839024b813648ce35d:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRjZWU1YzcxMzk5MmY0NzZmYmE3NWMzYjBiYjc2YWQ2YiQwYmE4YzMzMmRmYTk0NjM1YTFlODgyMmE5Y2Y0MzM3NQ=="
    MY_API_KEY = "OFRGNFhKc0JWUkxJc0ctNnhCNDg6X2tmZzdSR0pNWUN5OW9FSzYyVmtJdw=="

    service = ProductSearchService(cloud_id=MY_CLOUD_ID, api_key=MY_API_KEY)

    # 2. ПОДГОТОВКА ИНДЕКСА (Это исправит ошибку BadRequestError)
    service.recreate_index()

    # 3. ЗАГРУЗКА ДАННЫХ
    print("Загрузка товаров...")
    products = [
        {"title": "Apple MacBook Pro 14", "description": "Ноутбук M3", "price": 250000, "category": "Laptop"},
        {"title": "Samsung Galaxy S24", "description": "Смартфон", "price": 100000, "category": "Smartphone"},
        {"title": "Samsung Monitor", "description": "Монитор 4K", "price": 30000, "category": "Monitor"},
        {"title": "Sony Headphones", "description": "Наушники", "price": 25000, "category": "Audio"}
    ]

    for p in products:
        service.index_product(p)

    # Принудительное обновление для тестов
    service.es.indices.refresh(index=service.index_name)

    # 4. ТЕСТ ПОИСКА
    print("\n--- Поиск 'ноутбук apple' ---")
    results = service.search(query="ноутбук apple")
    print(f"Найдено: {results['total']}")
    for r in results['results']:
        print(f"  [{r.score:.2f}] {r.title}")

    print("\n--- Категории ---")
    for c in results['facets']['categories']:
        print(f"  {c['name']}: {c['count']}")

    # 5. ТЕСТ АВТОДОПОЛНЕНИЯ
    # Ищем 'sam' -> должно найти Samsung Galaxy и Samsung Monitor
    prefix = "sam"
    print(f"\n--- Автодополнение для '{prefix}' ---")
    suggestions = service.autocomplete(prefix)

    if suggestions:
        for s in suggestions:
            print(f" -> {s}")
    else:
        print("Подсказок нет.")

✅ Сервис подключен к: instance-0000000001
♻️ Индекс 'shop_products' пересоздан с поддержкой автодополнения.
Загрузка товаров...

--- Поиск 'ноутбук apple' ---
Найдено: 1
  [3.05] Apple MacBook Pro 14

--- Категории ---
  Laptop: 1

--- Автодополнение для 'sam' ---
 -> Samsung Galaxy S24
 -> Samsung Monitor
